# Preparación del entorno de trabajo

In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [ ]:
FILE_PATH = "/gdrive/My Drive/Colab Notebooks/Master/Vision por Computador/Practica 1/" #Guillermo
#FILE_PATH = "" #Antonio

# Imports

In [ ]:
import glob
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

import pickle

from tensorflow.keras.layers import Input, Flatten, Reshape
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, LeakyReLU, Dense, Dropout
from tensorflow.keras import Model, losses
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.utils import plot_model
from keras.layers.merge import concatenate

from scipy.spatial import distance

## Descargar desde Google Drive

## Descargar desde OneDrive

In [ ]:
!curl 'https://upm365-my.sharepoint.com/personal/guillermo_iglesias_hernandez_alumnos_upm_es/_layouts/15/download.aspx?SourceUrl=%2Fpersonal%2Fguillermo%5Figlesias%5Fhernandez%5Falumnos%5Fupm%5Fes%2FDocuments%2FUniversidad%2FVisi%C3%B3n%20por%20computador%2Fbinary%5F60x80%2Epkl' \
  -H 'authority: upm365-my.sharepoint.com' \
  -H 'sec-ch-ua: "Google Chrome";v="95", "Chromium";v="95", ";Not A Brand";v="99"' \
  -H 'sec-ch-ua-mobile: ?0' \
  -H 'sec-ch-ua-platform: "Windows"' \
  -H 'upgrade-insecure-requests: 1' \
  -H 'user-agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36' \
  -H 'accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9' \
  -H 'service-worker-navigation-preload: true' \
  -H 'sec-fetch-site: same-origin' \
  -H 'sec-fetch-mode: navigate' \
  -H 'sec-fetch-dest: iframe' \
  -H 'referer: https://upm365-my.sharepoint.com/personal/guillermo_iglesias_hernandez_alumnos_upm_es/_layouts/15/onedrive.aspx?id=%2Fpersonal%2Fguillermo%5Figlesias%5Fhernandez%5Falumnos%5Fupm%5Fes%2FDocuments%2FUniversidad%2FVisi%C3%B3n%20por%20computador%2Fbinary%5F60x80%2Epkl&parent=%2Fpersonal%2Fguillermo%5Figlesias%5Fhernandez%5Falumnos%5Fupm%5Fes%2FDocuments%2FUniversidad%2FVisi%C3%B3n%20por%20computador' \
  -H 'accept-language: es-ES,es;q=0.9,en;q=0.8' \
  -H 'cookie: MicrosoftApplicationsTelemetryDeviceId=51ca1bee-08da-404d-804f-935c15cdafaf; MicrosoftApplicationsTelemetryFirstLaunchTime=2021-11-04T08:48:57.393Z; rtFa=ylCdP6JhaymSaNjvza7EmitBKcYsz13R0J58O1+gU88mNkFGRUE4NUQtQzMyMy00MjcwLUI2OUQtQTRGQjM5MjdDMjU0IzEzMjgwNDMzNjM4NjY5NjY2MiNERjhGRkY5Ri0wMENBLTMwMDAtMzAwQy1GNDhCMjZEMjQ1NkQjR1VJTExFUk1PLklHTEVTSUFTLkhFUk5BTkRFWiU0MEFMVU1OT1MuVVBNLkVTnq3lJpwajWceYQWiVk2ivebguy3m+rUbFnMvxgcwiLsNdFOiv+8w6VWX0t9Tx/wORjvIA25y1FdNlqK13SPFmmfu9T6gZhGBt8W0Hu86KDmqi3k7ZHZ3fr+ObRY8wXZEYgCVO65S3JV2DWTqBDMoa4mOS+A/8eeZZs8muh/xXp+4YhPHhSWrrup9hiV2T3chq5yxnGx8pL67SAOzWMCNyhRLN0D+bO9961znRuIsnCCjSLSKie9qD1gXVlg3YJkd6eGn51q/jRMOQTUognub/A6UNQioB2Pv+zV6tMzCbFtu27/C9IKvRbFu5+WSnKOoHwheKzaYMler9abzTPxL+asAAAA=; FedAuth=77u/PD94bWwgdmVyc2lvbj0iMS4wIiBlbmNvZGluZz0idXRmLTgiPz48U1A+VjExLDBoLmZ8bWVtYmVyc2hpcHwxMDAzN2ZmZWE2MmU3NTkyQGxpdmUuY29tLDAjLmZ8bWVtYmVyc2hpcHxndWlsbGVybW8uaWdsZXNpYXMuaGVybmFuZGV6QGFsdW1ub3MudXBtLmVzLDEzMjgwNDMzNjM0MDAwMDAwMCwxMzE2NTQ5MTM5NjAwMDAwMDAsMTMyODA4NjU2Mzg2NTQxMDE0LDc5LjE1My4xMjMuMjI5LDMsNmFmZWE4NWQtYzMyMy00MjcwLWI2OWQtYTRmYjM5MjdjMjU0LCw2NTY0NTM1OS1lZGEwLTQwYWItOGYyNC1kNzQxZWMzYmU5MGEsZGY4ZmZmOWYtMDBjYS0zMDAwLTMwMGMtZjQ4YjI2ZDI0NTZkLGRmOGZmZjlmLTAwY2EtMzAwMC0zMDBjLWY0OGIyNmQyNDU2ZCwsMCwxMzI4MDUyMDAzODYyMjc4NzMsMTMyODA2OTI4Mzg2MjI3ODczLCwsZXlKNGJYTmZZMk1pT2lKYlhDSkRVREZjSWwwaUxDSjRiWE5mYzNOdElqb2lNU0lzSW5CeVpXWmxjbkpsWkY5MWMyVnlibUZ0WlNJNkltZDFhV3hzWlhKdGJ5NXBaMnhsYzJsaGN5NW9aWEp1WVc1a1pYcEFZV3gxYlc1dmN5NTFjRzB1WlhNaWZRPT0sMjY1MDQ2Nzc0Mzk5OTk5OTk5OSwxMzI4MDQzMzYzODAwMDAwMDAsMGRmNWJjYzYtZWFlMy00MzM0LWFkYzUtNzQ5Mzg0NDNiYTQ1LCwsLCwsMCxnL0NJeXZRZVNJbEQ1cFN3akI1NHBTQjNkaEwyVHRYZ0VWdFVOTW5rcHZIMmdYY04rcmZLdk1WaDQ3MTlHamJRTGFVdHNJRndRR0hOYTJLaStqWU1jTmFWOXRvcVhONUVGK1kzY3hTckZ1aHcrVjRITmxIaEFKdW0rYnltMXlOZzB3K202anRER1VtM1FZS25zZDN5UnhueFdQT3NVV3ZVN1AxVUI4QXVUL2pxeTFoT084V2ZoL2phd2g2Wmk2Z0tkNlFtVnZSbFM3RGFLZUx3ZFZWL2F1b2l5emhYMVUvMVA2b0FEMEhyTGluNG1YS2U4SThuakZ3YTUvbnpPK2FXdnREVzZqT2s1S29TQjdaS0d2TnpZUWRUWmNEYUhTUHJqeHlEWlg1NVcwUWd0ekdCNWNRenVrWENsNFZ3ZEExYUZxS3dqUFBDWDdMTm5rK2NiTUtoanc9PTwvU1A+; cucg=1; KillSwitchOverrides_enableKillSwitches=; KillSwitchOverrides_disableKillSwitches=' \
  -H 'if-none-match: "{1D67BB34-8728-4348-AD13-6EE348ACDEE0},2"' \
  --compressed --output 'binary_60x80.pkl'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
 22 2438M   22  559M    0     0   100M      0  0:00:24  0:00:05  0:00:19  116M^C


# Carga de los datos

In [ ]:
with open('/content/binary_60x80.pkl', 'rb') as file:
    X = pickle.load(file)

In [ ]:
X.shape

(44400, 80, 60, 3)

# Creación del autoencoder

In [ ]:
def create_autoencoder(X):
    input_encoder = Input(shape=(80, 60, 3), name='encoder_input')

    x = Conv2D(filters=32, kernel_size=(3,3), strides=(1,1), padding='same')(input_encoder)
    x = LeakyReLU(alpha=0.2)(x)
    x = Dropout(0.2)(x)
    x = Conv2D(filters=32, kernel_size=(3,3), strides=(2,2), padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Dropout(0.2)(x)
    assert tuple(x.shape.as_list()) == (None, 40, 30, 32)

    x = Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Dropout(0.2)(x)
    x = Conv2D(filters=64, kernel_size=(3,3), strides=(2,2), padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Dropout(0.2)(x)
    assert tuple(x.shape.as_list()) == (None, 20, 15, 64)

    x = Conv2D(filters=128, kernel_size=(3,3), strides=(1,1), padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Dropout(0.2)(x)
    x = Conv2D(filters=128, kernel_size=(3,3), strides=(2,2), padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Dropout(0.2)(x)
    assert tuple(x.shape.as_list()) == (None, 10, 8, 128)

    x = Flatten()(x)
    x = Dense(256)(x)
    assert tuple(x.shape.as_list()) == (None, 256)

    x = Dense(20)(x) # Capa de salida del encoder con X neuronas que corresponden a la representación de cada imagen
    output_encoder = LeakyReLU(alpha=0.2, name='encoder_output')(x)

    input_decoder = Dense(64*20*15, name='decoder_input')(output_encoder)
    x = LeakyReLU(alpha=0.2)(input_decoder)
    x = Dropout(0.2)(x)
    x = Reshape((20, 15, 64))(x)
    assert tuple(x.shape.as_list()) == (None, 20, 15, 64)

    x = Conv2DTranspose(filters=64, kernel_size=(3,3), strides=(1,1), padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Dropout(0.2)(x)
    x = Conv2DTranspose(filters=64, kernel_size=(3,3), strides=(2,2), padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Dropout(0.2)(x)
    assert tuple(x.shape.as_list()) == (None, 40, 30, 64)

    x = Conv2DTranspose(filters=32, kernel_size=(3,3), strides=(1,1), padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Dropout(0.2)(x)
    x = Conv2DTranspose(filters=32, kernel_size=(3,3), strides=(2,2), padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Dropout(0.2)(x)
    assert tuple(x.shape.as_list()) == (None, 80, 60, 32)

    output_decoder = Conv2DTranspose(filters=3, activation='sigmoid', kernel_size=(1,1), padding='same')(x)

    autoencoder = Model(input_encoder, output_decoder)
    encoder = Model(input_encoder, output_encoder)

    return encoder, autoencoder

In [ ]:
encoder, autoencoder = create_autoencoder(X)

## Ploteo de la arquitectura

In [ ]:
plot_model(encoder, show_layer_names=True, show_shapes=True)

In [ ]:
plot_model(autoencoder, show_layer_names=True, show_shapes=True)

## Entrenamiento del modelo

In [ ]:
def plot_train_val(history):
    plt.rcParams['figure.figsize'] = [15, 5]
    f, (ax1, ax2) = plt.subplots(1, 2, sharex=True)
    
    ax1.plot(history.history['loss'], label='loss (train)')
    ax1.set_title('Loss')
    ax1.set_xlabel('epoch')
    ax1.legend(loc="upper right")
    ax2.plot(history.history['mae'], label='mae (train)')
    ax2.set_title('MAE')
    ax2.set_xlabel('epoch')
    ax2.legend(loc="upper right")
    ax1.grid()
    ax2.grid()
    plt.plot()

In [ ]:
with tf.device('/device:GPU:0'):
    tf.keras.backend.clear_session()
    autoencoder.compile(
        loss='mse', 
        optimizer=Adam(learning_rate=0.002, beta_1=0.5), metrics=['mae'])
    history = autoencoder.fit(X, X,
                        epochs=50, batch_size=64, verbose=1)

## Evaluación del entrenamiento

In [ ]:
plot_train_val(history)

In [ ]:
reconstructed_img_predictions = autoencoder.predict(X)

In [ ]:
img_reconstructed_index = 50

plt.imshow(X[img_reconstructed_index])
plt.show()
plt.imshow(reconstructed_img_predictions[img_reconstructed_index])
plt.show()

# Predicción del sistema de recomendación

Se predicen las 10 características que resumen a cada imagen haciendo uso del predict del encoder. De esta manera se sucede la reducción dimensional

In [ ]:
encoder_predictions = encoder.predict(X)

In [ ]:
encoder_predictions.shape

## Función para recomendar imágenes

In [ ]:
def find_similar(index_img, encoder_predictions):
    img_searcher = encoder_predictions[index_img]
    img_finded = None
    img_finded_index = None
    distance_min = 99999999

    for index, candidate in enumerate(encoder_predictions):
        if index != index_img:
            distance_candidate = distance.euclidean(img_searcher, candidate)
            if distance_candidate < distance_min:
                distance_min = distance_candidate
                img_searcher = candidate
                img_finded_index = index

    return img_finded_index, distance_min

In [ ]:
img_search_index = 14
candidate, distance_candidate = find_similar(img_search_index, encoder_predictions)

print("-" * 10)
print("Found candidate " + str(candidate))
print("Distance: " + str(distance_candidate))

plt.imshow(X[img_search_index])
plt.show()
plt.imshow(X[candidate])
plt.show()